<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202302/blob/main/Lecture01/Notebook_SS01.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Regresión Lineal para predicción

Modelo 

$$
y= f(X) + u
$$

$$
y= X\beta + u
$$


Queremos predecir: $Y_i$

> *Ejemplo:* el logaritmo del salario

Características (aka **predictores**, **features**): $X_i=\left(X_{1i},\ldots,X_{pi}\right)'$

> *Ejemplo:* educación, edad, educación de los padres, habilidad cognitiva, etc.


In [ ]:
# Cargamos los paquetes

require("pacman")

p_load("tidyverse","stargazer")


In [ ]:
nlsy = read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/nlsy97.csv')

#Housekeeping
nlsy = nlsy  %>%   drop_na(educ) #dropea los valores faltantes (NA)

In [ ]:
head(nlsy)

## Regresión Lineal: log(salario) en polinomios de educación

In [ ]:
#Polinomios de educación

nlsy <- nlsy  %>% mutate(educ2=educ^2,
                         educ3=educ^3,
                         educ4=educ^4,
                         educ5=educ^5,
                         educ6=educ^6,
                         educ7=educ^7,
                         educ8=educ^8,
                         educ9=educ^9,
                         educ10=educ^10
                        )

In [ ]:
head(nlsy)

### Corremos la regresión

In [ ]:
reg<- lm(lnw_2016~educ+ educ2 + educ3 + educ4 + educ5 + educ6 + educ7 + 
    educ8 + educ9 + educ10,tol = 1e-9,   data = nlsy)


In [ ]:
summary(reg) #ver los resultados

In [ ]:
stargazer(reg,type="text") #Similar en stata: outreg2 

## Predicción

$$
\hat{y} = \hat{\beta}_0 +  \hat{\beta}_1 educ + .... +  \hat{\beta}_{10} educ^{10}
$$

In [ ]:
#predict me simplifica hacer la operación X\hat{\beta}

nlsy <- nlsy  %>% mutate(yhat=predict(reg)) #con la función mutate
#nlsy$yhat = predict(reg) #R base


In [ ]:
head(nlsy)

In [ ]:
# Voy a calcular E(y|X=x) y E(\hat(y)| X=x)

summ = nlsy %>%  
  group_by(
    educ, educ2, educ3, educ4, educ5, 
    educ6, educ7, educ8, educ9, educ10
  ) %>%  
  summarize(
    mean_y = mean(lnw_2016),
    yhat_reg = mean(yhat), .groups="drop"
  ) 

In [ ]:
head(summ)

In [ ]:
# graficamos
ggplot(summ) + 
  geom_point(
    aes(x = educ, y = mean_y),
    color = "blue", size = 2
  ) + 
  geom_line(
    aes(x = educ, y = yhat_reg), 
    color = "green", linewidth = 1.5
  ) + 
  labs(
    title = "ln Wages by Education in the NLSY",
    x = "Years of Schooling",
    y = "ln Wages"
  ) +
  theme_bw()


Como podemos ver, la regresión lineal de mínimos cuadrados puede aproximar cualquier función continua y ciertamente puede usarse para la predicción. 

Incluyendo un conjunto lo suficientemente rico de transformaciones, las predicciones de OLS producirán estimaciones insesgadas del verdadero predictor ideal: la función de expectativa condicional. Pero estas estimaciones serán bastante ruidosas.